# Determinar todos los compuestos dentro de las carpetas
<hr>

In [1]:
import glob
import pandas as pd

In [2]:
carpeta_bases = "Bases_conjuntas/Targets"
carpeta_inhibidores = "Inhibidores/Targets"

In [3]:
Csv_bases = glob.glob(f"{carpeta_bases}/**/*.csv", recursive=True)

In [4]:
Csv_inhibidores = glob.glob(f"{carpeta_inhibidores}/**/*.csv", recursive=True)

# Crear dataframes con los SMILES y la respectiva ruta
<hr>

In [6]:
Csv_bases

['Bases_conjuntas/Targets/EGFR_MAPK1/Mejores_predictivos_EGFR_MAPK1.csv',
 'Bases_conjuntas/Targets/EGFR_SRC/Mejores_predictivos_EGFR_SRC.csv',
 'Bases_conjuntas/Targets/EGFR_STAT3/Mejores_predictivos_EGFR_STAT3.csv']

In [7]:
import os

lista_dfs = []

for i in Csv_bases:
    df = pd.read_csv(i, index_col=0)
    # Obtener solo la carpeta (eliminar el archivo) de la ruta i
    carpeta = os.path.dirname(i)
    df["ruta"] = carpeta
    lista_dfs.append(df)

df_bases = pd.concat(lista_dfs, ignore_index=True)


In [8]:
import os

lista_dfs = []

for i in Csv_inhibidores:
    df = pd.read_csv(i, index_col=0)
    # Obtener solo la carpeta (eliminar el archivo) de la ruta i
    carpeta = os.path.dirname(i)
    df["ruta"] = carpeta
    lista_dfs.append(df)

df_inhibidores = pd.concat(lista_dfs, ignore_index=True)


# Establecer nombres temporales
<hr>

In [9]:
df_bases["ID"] = df_bases["Targets"].str.replace(" ", "_", regex=True) + "_" + df_bases.index.astype(str)


In [10]:
df_inhibidores["ID"] = df_inhibidores["Targets"].str.replace(" ", "_", regex=True) + "_" + df_inhibidores.index.astype(str)

In [12]:
df_bases.to_csv("IDS/df_bases.csv", index=False)

In [13]:
df_inhibidores.to_csv("IDS/df_inhibidores.csv", index=False)

# Preparar moléculas con obabel
<hr>

In [10]:
import os
import subprocess
from rdkit import Chem
from rdkit.Chem import AllChem

ruta_pythonsh = "/home/eliud7720/mgltools/mgltools_x86_64Linux2_1.5.7/bin/pythonsh"
ruta_prepare_ligand4 = "/home/eliud7720/mgltools/mgltools_x86_64Linux2_1.5.7/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_ligand4.py"

def smiles_to_pdb(smiles, pdb_filename):
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)

    result = AllChem.EmbedMolecule(mol, randomSeed=0xf00d)

    if result != 0:
        print(f"❌ No se pudo generar conformación 3D para: {smiles}")
        return  # O podrías lanzar un ValueError

    AllChem.MMFFOptimizeMolecule(mol)
    
    w = Chem.rdmolfiles.PDBWriter(pdb_filename)
    w.write(mol)
    w.close()


def preparar_ligando_con_mgltools(pdb_file, pdbqt_file, ruta_pythonsh, ruta_prepare_ligand4):
    directorio = os.path.dirname(pdb_file)  # Carpeta del archivo pdb
    
    cmd = [
        ruta_pythonsh,
        ruta_prepare_ligand4,
        "-l", os.path.basename(pdb_file),  # Solo el nombre para que busque en cwd
        "-o", os.path.basename(pdbqt_file),
        "-A", "hydrogens"
    ]
    
    # Ejecutar en el directorio donde están los archivos
    result = subprocess.run(cmd, cwd=directorio, capture_output=True, text=True)
    print("STDOUT:\n", result.stdout)
    print("STDERR:\n", result.stderr)
    
    if result.returncode != 0:
        print(f"Error preparando ligando para {pdb_file}")
        return False
    
    # Ahora sí eliminar el pdb original
    os.remove(pdb_file)
    return True



In [ ]:
longitud = len(df_bases)

for i, (nombre, smiles, ruta) in enumerate(zip(df_bases["ID"], df_bases["SMILES"], df_bases["ruta"]), 1):
    
    print(f"Procesando {i} de {longitud}")
    
    nombre_final_pdb = os.path.abspath(os.path.join(ruta, nombre + ".pdb"))
    nombre_final_pdbqt = os.path.abspath(os.path.join(ruta, nombre + ".pdbqt"))

    print(nombre_final_pdbqt)
    smiles_to_pdb(smiles, nombre_final_pdb)
    preparar_ligando_con_mgltools(nombre_final_pdb, nombre_final_pdbqt, ruta_pythonsh, ruta_prepare_ligand4)

Procesando 1 de 1151
/mnt/c/Users/Eliud/OneDrive/Escritorio/Docking/Compuestos/Bases_conjuntas/Targets/EGFR_MAPK1/EGFR_MAPK1_0.pdbqt
STDOUT:
 setting PYTHONHOME environment

STDERR:
 
Procesando 2 de 1151
/mnt/c/Users/Eliud/OneDrive/Escritorio/Docking/Compuestos/Bases_conjuntas/Targets/EGFR_MAPK1/EGFR_MAPK1_1.pdbqt
STDOUT:
 setting PYTHONHOME environment

STDERR:
 
Procesando 3 de 1151
/mnt/c/Users/Eliud/OneDrive/Escritorio/Docking/Compuestos/Bases_conjuntas/Targets/EGFR_MAPK1/EGFR_MAPK1_2.pdbqt
STDOUT:
 setting PYTHONHOME environment

STDERR:
 
Procesando 4 de 1151
/mnt/c/Users/Eliud/OneDrive/Escritorio/Docking/Compuestos/Bases_conjuntas/Targets/EGFR_MAPK1/EGFR_MAPK1_3.pdbqt
STDOUT:
 setting PYTHONHOME environment

STDERR:
 
Procesando 5 de 1151
/mnt/c/Users/Eliud/OneDrive/Escritorio/Docking/Compuestos/Bases_conjuntas/Targets/EGFR_MAPK1/EGFR_MAPK1_4.pdbqt
STDOUT:
 setting PYTHONHOME environment

STDERR:
 
Procesando 6 de 1151
/mnt/c/Users/Eliud/OneDrive/Escritorio/Docking/Compuestos/B

[12:16:38] UFFTYPER: Unrecognized charge state for atom: 8


STDOUT:
 setting PYTHONHOME environment

STDERR:
 
Procesando 958 de 1151
/mnt/c/Users/Eliud/OneDrive/Escritorio/Docking/Compuestos/Bases_conjuntas/Targets/EGFR_MAPK1/EGFR_MAPK1_957.pdbqt
STDOUT:
 setting PYTHONHOME environment

STDERR:
 
Procesando 959 de 1151
/mnt/c/Users/Eliud/OneDrive/Escritorio/Docking/Compuestos/Bases_conjuntas/Targets/EGFR_MAPK1/EGFR_MAPK1_958.pdbqt
STDOUT:
 setting PYTHONHOME environment

STDERR:
 
Procesando 960 de 1151
/mnt/c/Users/Eliud/OneDrive/Escritorio/Docking/Compuestos/Bases_conjuntas/Targets/EGFR_MAPK1/EGFR_MAPK1_959.pdbqt
STDOUT:
 setting PYTHONHOME environment

STDERR:
 
Procesando 961 de 1151
/mnt/c/Users/Eliud/OneDrive/Escritorio/Docking/Compuestos/Bases_conjuntas/Targets/EGFR_MAPK1/EGFR_MAPK1_960.pdbqt
STDOUT:
 setting PYTHONHOME environment

STDERR:
 
Procesando 962 de 1151
/mnt/c/Users/Eliud/OneDrive/Escritorio/Docking/Compuestos/Bases_conjuntas/Targets/EGFR_MAPK1/EGFR_MAPK1_961.pdbqt
STDOUT:
 setting PYTHONHOME environment

STDERR:
 
Procesand

In [11]:
longitud = len(df_inhibidores)

for i, (nombre, smiles, ruta) in enumerate(zip(df_inhibidores["ID"], df_inhibidores["SMILES"], df_inhibidores["ruta"]), 1):
    
    print(f"Procesando {i} de {longitud}")
    
    nombre_final_pdb = os.path.abspath(os.path.join(ruta, nombre + ".pdb"))
    nombre_final_pdbqt = os.path.abspath(os.path.join(ruta, nombre + ".pdbqt"))

    print(nombre_final_pdbqt)
    smiles_to_pdb(smiles, nombre_final_pdb)
    preparar_ligando_con_mgltools(nombre_final_pdb, nombre_final_pdbqt, ruta_pythonsh, ruta_prepare_ligand4)

Procesando 1 de 174
/mnt/c/Users/Eliud/OneDrive/Escritorio/Docking/Compuestos/Inhibidores/Targets/EGFR/MAPK1_P_EGFR_0.pdbqt
STDOUT:
 setting PYTHONHOME environment

STDERR:
 
Procesando 2 de 174
/mnt/c/Users/Eliud/OneDrive/Escritorio/Docking/Compuestos/Inhibidores/Targets/EGFR/MAPK1_P_EGFR_1.pdbqt
STDOUT:
 setting PYTHONHOME environment

STDERR:
 
Procesando 3 de 174
/mnt/c/Users/Eliud/OneDrive/Escritorio/Docking/Compuestos/Inhibidores/Targets/EGFR/MAPK1_P_EGFR_2.pdbqt
STDOUT:
 setting PYTHONHOME environment

STDERR:
 
Procesando 4 de 174
/mnt/c/Users/Eliud/OneDrive/Escritorio/Docking/Compuestos/Inhibidores/Targets/EGFR/MAPK1_P_EGFR_3.pdbqt
STDOUT:
 setting PYTHONHOME environment

STDERR:
 
Procesando 5 de 174
/mnt/c/Users/Eliud/OneDrive/Escritorio/Docking/Compuestos/Inhibidores/Targets/EGFR/MAPK1_P_EGFR_4.pdbqt
STDOUT:
 setting PYTHONHOME environment

STDERR:
 
Procesando 6 de 174
/mnt/c/Users/Eliud/OneDrive/Escritorio/Docking/Compuestos/Inhibidores/Targets/EGFR/MAPK1_P_EGFR_5.pdbqt
S